In [2]:
import pandas as pd
from fbprophet import Prophet

In [3]:
df = pd.read_excel("Sales workbook.xlsx")

In [4]:
df.head()

,Date,ITEM NAME,QTY,B.P/UNIT,TOTAL B.P,TOTAL S.P,PROFIT,CATEGORY
0,2017 SEPTEMBER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-09-23 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin
2,2017-09-23 00:00:00,Nursing Cover,1.0,636.0,636.0,1500.0,864,Feeding & Nursing
3,2017 OCTOBER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-10-11 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin


In [5]:
df = df[pd.notnull(df['ITEM NAME'])]

In [6]:
df.head()

,Date,ITEM NAME,QTY,B.P/UNIT,TOTAL B.P,TOTAL S.P,PROFIT,CATEGORY
1,2017-09-23 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin
2,2017-09-23 00:00:00,Nursing Cover,1.0,636.0,636.0,1500.0,864,Feeding & Nursing
4,2017-10-11 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin
5,2017-10-16 00:00:00,Changing Station,1.0,1378.0,1378.0,3000.0,1622,Diapering & Potty
6,2017-10-16 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin


In [7]:
pd.to_datetime(df['Date'])

1     2017-09-23
2     2017-09-23
4     2017-10-11
5     2017-10-16
6     2017-10-16
         ...    
397   2019-07-22
398   2019-07-23
399   2019-07-23
400   2019-07-23
401   2019-07-23
Name: Date, Length: 383, dtype: datetime64[ns]

In [24]:
df['ds'] = pd.to_datetime(df['Date']).apply(lambda x : x.replace(day=1))

In [26]:
df.head()

,Date,ITEM NAME,QTY,B.P/UNIT,TOTAL B.P,TOTAL S.P,PROFIT,CATEGORY,month,ds
1,2017-09-23 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-09-01,2017-09-01
2,2017-09-23 00:00:00,Nursing Cover,1.0,636.0,636.0,1500.0,864,Feeding & Nursing,2017-09-01,2017-09-01
4,2017-10-11 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-10-01,2017-10-01
5,2017-10-16 00:00:00,Changing Station,1.0,1378.0,1378.0,3000.0,1622,Diapering & Potty,2017-10-01,2017-10-01
6,2017-10-16 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-10-01,2017-10-01


In [37]:
df.rename(columns={"QTY": "y"},inplace=True)

In [38]:
df.head()

,Date,ITEM NAME,y,B.P/UNIT,TOTAL B.P,TOTAL S.P,PROFIT,CATEGORY,month,ds
1,2017-09-23 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-09-01,2017-09-01
2,2017-09-23 00:00:00,Nursing Cover,1.0,636.0,636.0,1500.0,864,Feeding & Nursing,2017-09-01,2017-09-01
4,2017-10-11 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-10-01,2017-10-01
5,2017-10-16 00:00:00,Changing Station,1.0,1378.0,1378.0,3000.0,1622,Diapering & Potty,2017-10-01,2017-10-01
6,2017-10-16 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-10-01,2017-10-01


In [19]:
items = pd.DataFrame()

In [21]:
items['ITEM NAME'] = df['ITEM NAME']

In [22]:
items['ITEM NAME'].head()

1         Washclothes
2       Nursing Cover
4         Washclothes
5    Changing Station
6         Washclothes
Name: ITEM NAME, dtype: object

In [30]:
items['ITEM NAME'] = items.drop_duplicates('ITEM NAME', keep='first')

In [39]:
def ComputeValue(item):
  #create monthly sales of an item
  sales = df[df['ITEM NAME'] == item]
  sales = sales.groupby(['ds'])['y'].sum().reset_index(name='y')

    
  if(sales['ds'].count() > 1):
    #predict sales
    print(item)
    model = Prophet(seasonality_mode='multiplicative') #instantiate Prophet
    model.fit(sales); #fit the model with your dataframe

    future_data = model.make_future_dataframe(periods=24,freq='M')
    forecast_data = model.predict(future_data)

    predicted_lower = int(forecast_data[forecast_data['ds'] == '2019-07-31'].yhat_lower)
    
    predicted = int(forecast_data[forecast_data['ds'] == '2019-07-31'].yhat)
    
    predicted_upper = int(forecast_data[forecast_data['ds'] == '2019-07-31'].yhat_upper)
    
    xdict = pd.Series([item, predicted_lower,predicted,predicted_upper], index=['itemname','predicted_lower','pred','predicted_upper'])
  else:
    xdict = pd.Series([item,0,0,0], index=['itemname','predicted_lower','pred','predicted_upper'])
  return xdict

In [62]:
mydict = items.apply(lambda x : ComputeValue(x['ITEM NAME']), axis=1)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 3.


Washclothes


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 3.


Washclothes


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 2.


...

5 in 1 Rocker Bassinet


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 0.


Winkel Activity Toy


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 0.


Changing Clutch


In [63]:
mydict.head()

,itemname,predicted_lower,pred,predicted_upper
1,Washclothes,0.0,0.0,1.0
2,Nursing Cover,0.0,1.0,1.0
4,NaN,0.0,0.0,0.0
5,Changing Station,0.0,1.0,1.0
6,NaN,0.0,0.0,0.0


In [64]:
mydict = mydict[pd.notnull(mydict['itemname'])]

In [65]:
mydict.head()

,itemname,predicted_lower,pred,predicted_upper
1,Washclothes,0.0,0.0,1.0
2,Nursing Cover,0.0,1.0,1.0
5,Changing Station,0.0,1.0,1.0
13,4 - Pack Longsleeve bodysuits,0.0,0.0,0.0
15,Neon swaddle blankets,0.0,0.0,0.0


In [46]:
mydict.to_csv("predicted items.csv")

In [66]:
mydict['ds'] = '2019-08-01'

In [67]:
mydict.head()

,itemname,predicted_lower,pred,predicted_upper,ds
1,Washclothes,0.0,0.0,1.0,2019-08-01
2,Nursing Cover,0.0,1.0,1.0,2019-08-01
5,Changing Station,0.0,1.0,1.0,2019-08-01
13,4 - Pack Longsleeve bodysuits,0.0,0.0,0.0,2019-08-01
15,Neon swaddle blankets,0.0,0.0,0.0,2019-08-01


In [68]:
mydict.rename(columns={"predicted_upper": "y"},inplace=True)

In [69]:
mydict.head()

,itemname,predicted_lower,pred,y,ds
1,Washclothes,0.0,0.0,1.0,2019-08-01
2,Nursing Cover,0.0,1.0,1.0,2019-08-01
5,Changing Station,0.0,1.0,1.0,2019-08-01
13,4 - Pack Longsleeve bodysuits,0.0,0.0,0.0,2019-08-01
15,Neon swaddle blankets,0.0,0.0,0.0,2019-08-01


In [70]:
mydict.drop(['predicted_lower','pred'],axis=1,inplace=True)

In [71]:
mydict.head()

,itemname,y,ds
1,Washclothes,1.0,2019-08-01
2,Nursing Cover,1.0,2019-08-01
5,Changing Station,1.0,2019-08-01
13,4 - Pack Longsleeve bodysuits,0.0,2019-08-01
15,Neon swaddle blankets,0.0,2019-08-01


In [73]:
mydict.rename(columns={"itemname": "ITEM NAME"},inplace=True)

In [72]:
df.head()

,Date,ITEM NAME,y,B.P/UNIT,TOTAL B.P,TOTAL S.P,PROFIT,CATEGORY,month,ds
1,2017-09-23 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-09-01,2017-09-01
2,2017-09-23 00:00:00,Nursing Cover,1.0,636.0,636.0,1500.0,864,Feeding & Nursing,2017-09-01,2017-09-01
4,2017-10-11 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-10-01,2017-10-01
5,2017-10-16 00:00:00,Changing Station,1.0,1378.0,1378.0,3000.0,1622,Diapering & Potty,2017-10-01,2017-10-01
6,2017-10-16 00:00:00,Washclothes,1.0,530.0,530.0,1500.0,970,Bath & Skin,2017-10-01,2017-10-01


In [74]:
df1 = df[['ITEM NAME','y','ds']]

In [75]:
df1.head()

,ITEM NAME,y,ds
1,Washclothes,1.0,2017-09-01
2,Nursing Cover,1.0,2017-09-01
4,Washclothes,1.0,2017-10-01
5,Changing Station,1.0,2017-10-01
6,Washclothes,1.0,2017-10-01


In [77]:
df2 = df1.groupby(['ITEM NAME','ds'])['y'].sum().reset_index(name="y")

In [78]:
df2.head()

,ITEM NAME,ds,y
0,10 Pack Washclothes(Pink),2018-12-01,1.0
1,10-Pack Wash clothes,2018-12-01,1.0
2,10-Pack Washcloths,2019-07-01,1.0
3,10-pack Washclothes,2018-12-01,1.0
4,10-pack Washclothes(blue),2019-06-01,1.0


In [84]:
combined = pd.concat([df2,mydict],sort=False)

In [85]:
combined.to_csv("combined.csv")